# 20-1 파이썬으로 이미지 파일 다루기
색상 히스토그램을 기반으로 주어진 이미지들 중 비슷한 색상 분포를 가지고 있는 이미지를 찾아주는 기능을 구현

In [1]:
# 필요한 라이브러리를 설치 순서대로 Pillow, OpenCV, Matplotlib

# $ pip install pillow opencv-python matplotlib

# 설치 디렉토리 구성 및 실습용 이미지 파일 다운로드
# $ mkdir -p ~/aiffel/python_image_proc
# $ wget https://aiffelstaticprd.blob.core.windows.net/media/documents/pillow_practice.png
# $ mv pillow_practice.png ~/aiffel/python_image_proc

# 실습용으로 사용될 CIFAR-100 이미지 데이터셋 다운로드
# $ wget https://aiffelstaticprd.blob.core.windows.net/media/documents/cifar-100-python.tar.gz
# $ mv cifar-100-python.tar.gz ~/aiffel/python_image_proc
# $ cd ~/aiffel/python_image_proc && tar xvzf cifar-100-python.tar.gz

## 10-2 디지털 이미지
화소: 색상을 가지는 점 하나 / 화소는 RGB 세 개의 단일색의 강도를 각각 조절하여 색상 표현  
  
  
이미지 저장 방법 중 가장 단순한 방법은 각 점 하나하나의 색상 값을 지정하는 방식  
- 래스터(raster) 또는 비트맵(bitmap) 방식의 이미지
- 보통 한 점마다 각 색상별로 8비트를 사용, 0 ~ 255 사이의 값($2^8 = 256$)으로 해당 색의 감도 표시
  
벡터(vector) 방식의 이미지는 상대적의 점과 선의 위치를 방정식으로써 기록, 확대 및 축소에 따라 재계산하여 깨짐이 없다.  
  
우리가 주로 다루는 파일 중 사진 파일은 래스터 방식, 글꼴은 벡터 방식  
  
RGB 방식 이외에도 YUV(기존 흑백 채널에 1/4의 해상도를 가진 두 색상 채널 덧붙여 송출), HSV(Hue 색상, Saturation 채도, Value 명도), 인쇄: CMYK(Cyan, Magenta, Yellow, Black)의 방법이 있다.  
이처럼 색을 표현하는 다양한 방식을 컬러 스페이스(color space, 색 공간)라고 한다.  
채널: 각 컬러 스페이스를 구성하는 단일 축(RGB에서 각각의 R, G, B)  
  
  
JPEG 이미지 형식은 근처 화소를 묶어 비슷한 색들을 뭉뚱그리는 방식으로 이미지 압축(색상 정보 손실)
PNG 이미지 형식의 경우 색상의 손실 없이 이미지를 압축
GIF 형식의 이미지는 이미지 내에 여러 프레임을 두어 이를 움직이게 만들 수 있고, 색상 정보를 손실 없이 저장하지만, 256개의 색상만 기억할 수 있는 팔레트 방식으로 제한  

In [5]:
## Pillow 사용법
# PIL을 이용해 바로 이미지 하나를 직접 생성하기  
# 이미지는 배열 형태의 데이터  
# 만약 가로 세로 각 32픽셀에 RGB 세 가지 색상 채널 → Numpy로 [32, 32, 3] 차원의 배열을 생성  
# 데이터 타입은 uint8
# 각 값이 부호가 없는 8비트 정수가 되어 0~255(2^8 = 256) 사이의 값을 나타냄

import numpy as np
from PIL import Image

data = np.zeros([32,32,3], dtype = np.uint8)    # 배열 만들기
image = Image.fromarray(data, 'RGB')            # PIL.Image.fromarray() → 이미지 객체로 변환
image.show()                                    # 화면에 표시

# 32*32짜리 까만 이미지 화면이 뜬다.

In [6]:
#### 연습 문제
# 1. 가로 세로 각 128 픽셀짜리 흰색 이미지 만들어 화면에 표시

data = np.zeros([128,128,3], dtype=np.uint8)    # 배열 만들기
data[:,:] = [255,255,255]
image = Image.fromarray(data, 'RGB')
image.show()

In [7]:
# 2. 연습용 이미지를 열어 width와 height를 출력하고, .save()를 이용하여 jpg 파일 포맷으로 저장

from PIL import Image
#import os

# 연습용 파일 경로
#image_path = os.getenv('HOME')+'/aiffel/python_image_proc/pillow_practice.png'
image_path = './aiffel/python_image_proc/pillow_practice.png'    # PNG 파일

# 이미지 열기
img = Image.open(image_path)
img.show()

# width와 height 출력
print(img.width)
print(img.height)

# JPG 파일 형식으로 저장해보기
#new_image_path = os.getenv('HOME')+'/aiffel/python_image_proc/pillow_practice.jpg'
new_image_path = './aiffel/python_image_proc/pillow_practice.jpg'    # JPG 파일
img = img.convert('RGB')
img.save(new_image_path)    # new_image_path 경로에 jpg 이미지 저장하기

620
465


In [8]:
# 3. resize()를 이용하여 이미지 크기를 100*200으로 변경하여 저장하기

resized_image = img.resize((100,200))    # 이미지 크기 변경
resized_image.show()

# resized_image_path = os.getenv('HOME')+'/aiffel/python_image_proc/pillow_practice_resized.png'
resized_image_path = './aiffel/python_image_proc/pillow_practice_resized.png'
resized_image.save(resized_image_path)

In [9]:
# 4. .crop()을 이용하여 눈 부분만 잘라내어 저장하기

box = (300,100,600,400)
region = img.crop(box)
region.show()

# cropped_image_path = os.getenv('HOME')+'/aiffel/python_image_proc/pillow_practice_cropped.png'
cropped_image_path = './aiffel/python_image_proc/pillow_practice_cropped.png'
region.save(cropped_image_path)

In [1]:
## Pillow를 활용한 데이터 전처리
# CIFAR-100 데이터를 받아 개별 이미지 파일로 추출

#import os
import pickle

#dir_path = os.getenv('HOME')+'/aiffel/python_image_proc/cifar-100-python'
dir_path = './aiffel/python_image_proc/cifar-100-python'

#train_file_path = os.path.join(dir_path, 'train')
train_file_path = dir_path + '/train'
print(train_file_path)
with open(train_file_path, 'rb') as f:
    train = pickle.load(f, encoding='bytes')
    
print(type(train))
# print(train)

./aiffel/python_image_proc/cifar-100-python/train
<class 'dict'>


In [20]:
train.keys()

# 각 키들이 문자열이 아닌 b로 시작하는 bytes로 되어있다.

dict_keys([b'filenames', b'batch_label', b'fine_labels', b'coarse_labels', b'data'])

In [21]:
type(train[b'filenames'])

# list 형태이다.

list

In [22]:
train[b'filenames'][0:5]

[b'bos_taurus_s_000507.png',
 b'stegosaurus_s_000125.png',
 b'mcintosh_s_000643.png',
 b'altar_boy_s_001435.png',
 b'cichlid_s_000031.png']

In [23]:
train[b'data'][0:5]

# 파일 이름에 해당하는 이미지 위치(배열로 나왔다.)

array([[255, 255, 255, ...,  10,  59,  79],
       [255, 253, 253, ..., 253, 253, 255],
       [250, 248, 247, ..., 194, 207, 228],
       [124, 131, 135, ..., 232, 236, 231],
       [ 43,  32,  87, ...,  60,  29,  37]], dtype=uint8)

In [24]:
train[b'data'][0].shape

# 3072라는 숫자는 3채널 * 1024(32*32)를 의미 - 화소
# R(1024 + G(1024) + B(1024)
# numpy 배열을 잘 reshape하면 이미지 파일 원본이 복구된다.

(3072,)

In [26]:
# 모양만 맞추어 reshape하면 안되고, 1024를 32X32에 채우는 것을 3번 반복하는 방식

image_data = train[b'data'][0].reshape([32, 32, 3], order='F') # order 주의
# order='F' : 앞선 차원부터 데이터를 채우는 방식으로 reshape
image = Image.fromarray(image_data)
# Pillow를 사용하여 Numpy 배열을 Image객체로 만들어서
image.show() # 화면에 띄우기

# 이미지의 X축과 Y축이 뒤집어져 있다.

In [31]:
# 이미지 축 바꾸기

image_data = image_data.swapaxes(0, 1)    # swapaxes(0,1) 축 바꾸기
image = Image.fromarray(image_data)
image.show()

In [29]:
#import os
import pickle
from PIL import Image
import numpy
from tqdm import tqdm
#dir_path = os.getenv('HOME')+'/aiffel/python_image_proc/cifar-100-python'
dir_path = './aiffel/python_image_proc/cifar-100-python'
train_file_path = os.path.join(dir_path, 'train')

# image를 저장할 cifar-100-python의 하위 디렉토리(images)를 생성합니다.
# images_dir_path = os.path.join(dir_path, 'images')
images_dir_path = dir_path + '/images'

#if not os.path.exists(images_dir_path):
#    os.mkdir(images_dir_path) # images 디렉토리 생성

# 32X32의 이미지 파일 50000개를 생성합니다.
with open(train_file_path, 'rb') as f:
    train = pickle.load(f, encoding='bytes')
    
    for i in tqdm(range(len(train[b'filenames']))):
        filename = train[b'filenames'][i].decode()
        data = train[b'data'][i].reshape([32, 32, 3], order='F')
        image = Image.fromarray(data.swapaxes(0, 1))
        image.save(os.path.join(images_dir_path, filename))
        
# 폴더를 확인했더니 50000개의 이미지가 담긴 모습을 볼 수 있다.

100%|██████████████████████████████████████████████████████████████████████████| 50000/50000 [00:46<00:00, 1066.12it/s]
